[View in Colaboratory](https://colab.research.google.com/github/lewisliching/Fashion_MNIST_with_Keras/blob/master/Fashion_MNIST_with_Keras.ipynb)

##Set up Working environemnt of Google Colab and Google drive

- Suppose Tensorflow is ready
- Install Keras such that Numpy....libraries are updated as well
- Install PyDrive for accessing Google drive and authorize Google Colab to access files

In [7]:
!pip install keras
!pip install -U -q PyDrive

##Connect Google drive for CSV files

- Authenticate and create the PyDrive client and it will prompt for unique key of verify Google Colab can have access to Google Drive. Just click and copy the unique key and copy to the blank

- Copy the Folder ID from URL bar of that folder

- All files in that folder are listed with file ID in the format of title: xxx.csv, id: xxx

- Use the files ID to open and load files into Google colab and create filename accordingly

In [13]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# list out files in folder of Google drive
file_list = drive.ListFile({'q': "'1xt6pBbi0fLeFeUqc4OuwD0SciUad9jhf' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

# Load files into Google colab
train_downloaded = drive.CreateFile({'id': '1nX2Igu-Yyk6s0PzRa6HXmJnovq_lD3P9'})
train_downloaded.GetContentFile('fashion-mnist_train.csv')  
test_downloaded = drive.CreateFile({'id': '1BQt3R3xe83Iiw2GQx7z_PrALfAmD7d4M'})
test_downloaded.GetContentFile('fashion-mnist_test.csv')

print('Files are loaded into Colab')

title: fashion-mnist_test.csv, id: 1BQt3R3xe83Iiw2GQx7z_PrALfAmD7d4M
title: fashion-mnist_train.csv, id: 1nX2Igu-Yyk6s0PzRa6HXmJnovq_lD3P9
Files are loaded into Colab


##Import CSV file into DataFrame from Colab

- Using Pandas read_csv to read file in raw format

- Printout header (1st 5 row) of DataFrame for double checking

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

train_df = pd.read_csv(r'fashion-mnist_train.csv')
test_df = pd.read_csv(r'fashion-mnist_test.csv')

train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
